# Redes neuronales 2

Los clasificadores no solo trabajan sobre dos clases, es muy común encontrarnos con problemas con más de dos. Para adaptar una red neuronal a un problema de $n$ clases basta con colocar en la capa de salida $n$ neuronas. 

<img src="imgs/multiclass.svg" width="60%">

Vamos a estudiar esto haciendo uso de un *dataset* muy conocido utilizado para clasificar el género de plantas iris. La ﬂor de iris se presenta en tres especies distintas denominadas: setosa, versicolor y virgínica. Pueden identiﬁcarse atendiendo a los anchos y altos de sus pétalos y sépalos. Este conjunto de datos contiene 150 muestras de esta ﬂor, recogiendo 50 muestras de cada especie. Aquí tienes el [*dataset*](data/iris.data), descárgalo y échale un vistazo.


Las cuatro primeras columnas corresponden al alto y ancho del sépalo y al alto y ancho del pétalo. La quinta columna indica la especie a la que pertenece: 0 → setosa, 1 → versicolor y 2 → virgínica. 

<img src="imgs/iris.png" width="25%">


Vemos que la quinta columna contiene los valores 0,1 y 2. Son las etiquetas correspondientes a las tres clases. Decíamos que la forma de adaptar ahora una red para clasificar tres clases es poner en la capa de salida tres neuronas. Recordemos que la función de activación que estamos usando es la sigmoide, cuyo rango es $(0,1)$. ¿Cómo hacemos encajar esto para que podamos definir una función de error sobre la cual podamos descender por el gradiente hasta un mínimo?

### Codificación onehot

La solución está en utilizar la codificación **onehot**. Esta codificación hace lo siguiente, se toma un vector de una dimensión igual al número de clases que haya, en este caso, 3. Todas las componentes del vector estarán a 0, excepto una que tendrá el valor 1 y que corresponderá con la clase que defina. Es decir: 0 →[1,0,0], 1 →[0,1,0] y 2 →[0,0,1]. 



### Red de clasifiación

Definamos ahora la red que vamos a utilizar para nuestro clasificador. Se compondrá de: una capa de entrada con 4 entradas (ancho y alto de pétalo y sépalo), 5 neuronas en la capa oculta y 3 neuronas en la capa de salida, dado que hay tres clases. *¿Y por qué cinco neuronas en la capa oculta? -Porque lo digo yo, que para eso soy el profesor.-*

<img src="imgs/rediris.png" width="25%">


### Función de activación softmax

Habíamos dicho que cada neurona tiene como función de activación la función sigmoide. Vamos a cambiar esto para la **última, y solo para la última capa**. En la última capa quitamos la función sigmoide y colocamos la función **softmax**. 
La función softmax es parecida a la sigmoide, ésta también convierte cualquier valor a un valor entre cero y uno. La diferencia está en que la función softmax no trabaja sobre un valor sino sobre un vector. De esta forma, convierte todas las componentes de un vector en valores entre cero y uno, pero, además, garantiza que la suma de todos estos valores sea 1. 
>La salida de la función softmax conforma una **distribución de probabilidad**.

La expresión de la función softmax es:

$$ Softmax(\vec{v})_i = \frac{e^{v_i}}{\sum_{j=1}^{n}{e^{v_j}}} $$

Veamos esto con un poco de código:

In [20]:
import numpy as np

logits = np.array([8.34, -2.87, 0.002])  # Terminología: al valor de salida de una neurona antes de pasar por la función de activación se le denomina: "logit".

def softmax(i, v):
    return np.exp(v[i]) / np.sum(np.exp(v)) 
    
a = softmax(0, logits)
b = softmax(1, logits)
c = softmax(2, logits)

print(a,"+",b,"+",c,"=", a+b+c)
    

0.999747275384849 + 1.3534714121386372e-05 + 0.0002391899010295041 = 0.9999999999999999


### Función de error

Nuestra función de error va a comparar las salidas de la red con la etiqueta. Supongamos que nuestra red devuelve (salida del softmax) para una determinada entrada los valores: $o_1=0.78, o_2=0.03, o_3=0,19$, y que su correspondiente etiqueta es: $(1,0,0)$ en codificación softmax. Por tanto, el error, para esta muestra, será: 

$$error = (1-0.78)^2 + (0-0.03)^2 + (0-0.19)^2 = 0.0854$$

Pero, dado que tenemos 150 muestras, la función de error final la calcularíamos sumando los 150 errores particulares de cada muestra.


### Mini-batch

La forma de calcular la función de error que acabamos de ver no se hace así en la práctica. Lo entenderemos mejor con el siguiente símil. Supongamos que queremos saber la media de altura de todos los habitantes de España. Deberíamos medir a cada persona, sumar todas las medidas y dividir por el número total de personas. Esta es la media exacta, pero nos iba a costar mucho poder llevar a cabo todas estas medidas (somos algo más de 46 millones). Otra forma mucho más asequible sería tomar una muestra de esta población y calcular la media muestral. Si la muestra es suficientemente variada y amplia obtendremos un valor muy aproximado a la media real pero con mucho menos esfuerzo.

Algo parecido podemos aplicar sobre la función de error. Si, en lugar de utilizar todo el conjunto de datos para calcular el error y luego el gradiente que nos lleve a un nuevo conjunto de pesos y umbrales, tomamos una muestra variada de los datos para calcular el error, el gradiente que obtendremos será muy parecido y nos llevará mucho menos tiempo. Por lo tanto, podremos avanzar mucho más rápido descendiendo por el gradiente.

Al conjunto de muestras que tomamos del *dataset* lo llamaremos **mini-batch**. Por ejemplo, en el caso del *dataset* del iris, son 150 muestras, que, una vez bien “barajadas” para que la muestra sea variada y representativa, podremos tomar mini-lotes de 15 muestras. Teniendo un total de 10 mini-lotes. Con *datasets* pequeños como este no se aprecia visiblemente la mejora en velocidad, pero con conjuntos de decenas de miles de muestras o más, la reducción de velocidad es drástica.

Por tanto, iremos tomando sucesivos *mini-batchs* de forma iterativa hasta completar el número de estos. Cuando hayamos procesado todos los *mini-batchs* diremos que hemos completado una época (**epoch**), y volveremos a repetir todo el proceso hasta  aproximarnos suficientemente a un mínimo local de la función de error.
